In [ ]:
# ============================================================
# INFERENCIA "AUTO"(regresión / binaria / multiclase)
# Entradas:
#   1) resultados.zip   (contiene modelo.keras + metadata.json)
#   2) New_final.csv    (nuevos datos con las MISMAS features)
# Salida:
#   New_predicciones.csv (agrega columnas según el tipo de problema)
# ============================================================

import os
import json
import zipfile
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras

# ----------------------------
# 0) Config
# ----------------------------
ZIP_RESULTADOS = "resultados.zip"
NEW_CSV = "New_final.csv"
OUT_PRED = "New_predicciones.csv"

EXTRACT_DIR = "inferencia_artifacts"
os.makedirs(EXTRACT_DIR, exist_ok=True)

# ----------------------------
# 1) Descomprimir resultados
# ----------------------------
with zipfile.ZipFile(ZIP_RESULTADOS, "r") as z:
    z.extractall(EXTRACT_DIR)

model_path = os.path.join(EXTRACT_DIR, "modelo.keras")
meta_path  = os.path.join(EXTRACT_DIR, "metadata.json")

# ----------------------------
# 2) Cargar modelo + metadata
# ----------------------------
model = keras.models.load_model(model_path)

with open(meta_path, "r", encoding="utf-8") as f:
    meta = json.load(f)

n0_esperado = int(meta.get("n_features", -1))

# Umbral para binaria:
alpha = meta.get("alpha", None)
if alpha is None:
    alpha = meta.get("threshold", 0.5)
alpha = float(alpha)

# ¿clase positiva crítica?
clase_positiva_critica = meta.get("clase_positiva_critica", None)
if clase_positiva_critica is None:
    clases_simetricas = bool(meta.get("clases_simetricas", True))
    clase_positiva_critica = (not clases_simetricas)
else:
    clase_positiva_critica = bool(clase_positiva_critica)

metrica_principal = meta.get("metrica_principal", None)
if metrica_principal is None:
    metrica_principal = "Accuracy" if not clase_positiva_critica else "F1"
metrica_principal = str(metrica_principal)

# ----------------------------
# 3) Cargar nuevos datos
# ----------------------------
new = pd.read_csv(NEW_CSV)

# Si viene target, se quita para inferencia
if "target" in new.columns:
    new_X = new.drop(columns=["target"])
else:
    new_X = new.copy()

# ----------------------------
# 4) Validación mínima de columnas
# ----------------------------
if n0_esperado != -1 and new_X.shape[1] != n0_esperado:
    raise ValueError(
        f"New_final.csv tiene {new_X.shape[1]} features, pero el modelo espera {n0_esperado}. "
        "Asegúrate de aplicar el mismo preprocesamiento y tener las mismas columnas."
    )

non_numeric = [c for c in new_X.columns if not pd.api.types.is_numeric_dtype(new_X[c])]
if non_numeric:
    raise ValueError(
        f"Estas columnas no son numéricas: {non_numeric}. "
        "New_final.csv debe venir ya preprocesado (solo numérico)."
    )

X = new_X.astype(np.float32).values

# ----------------------------
# 5) Predicción cruda
# ----------------------------
y_raw = model.predict(X, verbose=0)

# ----------------------------
# 6) Detección automática del tipo de problema
# ----------------------------
out_shape = model.output_shape
out_dim = int(out_shape[-1]) if isinstance(out_shape, tuple) else int(out_shape[0][-1])

last_layer = model.layers[-1]
act_name = None
try:
    act = getattr(last_layer, "activation", None)
    act_name = act.__name__ if act is not None else None
except Exception:
    act_name = None

def detect_task(out_dim: int, act_name: str | None) -> str:
    act_name = (act_name or "").lower()
    if out_dim == 1:
        if "sigmoid" in act_name:
            return "binary_classification"
        return "regression"
    else:
        if "softmax" in act_name:
            return "multiclass_classification"
        return "multiclass_classification"

task = detect_task(out_dim, act_name)

print("=== Detección automática ===")
print("output_shape:", model.output_shape)
print("last_activation:", act_name)
print("task_detected:", task)

if task == "binary_classification":
    print("\n=== Configuración binaria desde metadata ===")
    print("Alpha empleado (umbral):", alpha)
    print("¿Clase positiva (1) crítica?:", clase_positiva_critica)
    print("Métrica principal:", metrica_principal)

# ----------------------------
# Helpers multiclase
# ----------------------------
def stable_softmax(z: np.ndarray) -> np.ndarray:
    # z: (n, C)
    z = z - np.max(z, axis=1, keepdims=True)
    ez = np.exp(z)
    return ez / np.sum(ez, axis=1, keepdims=True)

def get_index_to_class_from_meta(meta: dict, out_dim: int):
    """
    Devuelve:
      - index_to_class: dict[int,int|str] o None si no existe
      - classes_original: list en el orden interno 0..C-1 si se puede
    """
    index_to_class = meta.get("index_to_class", None)
    classes_original = None

    if isinstance(index_to_class, dict) and len(index_to_class) > 0:
        # keys vienen como str en json, los pasamos a int
        tmp = {}
        for k, v in index_to_class.items():
            try:
                ki = int(k)
            except Exception:
                continue
            tmp[ki] = v
        index_to_class = tmp

        # Armamos lista en orden 0..C-1 si están todos
        ok = all((i in index_to_class) for i in range(out_dim))
        if ok:
            classes_original = [index_to_class[i] for i in range(out_dim)]
        return index_to_class, classes_original

    # Fallback: si existe classes_original_train como lista y coincide con out_dim
    classes_original_train = meta.get("classes_original_train", None)
    if isinstance(classes_original_train, list) and len(classes_original_train) == out_dim:
        classes_original = classes_original_train
        index_to_class = {i: classes_original[i] for i in range(out_dim)}
        return index_to_class, classes_original

    return None, None

# ----------------------------
# 7) Post-procesamiento según tarea + guardar
# ----------------------------
out = new.copy()

if task == "regression":
    y_pred = np.asarray(y_raw).reshape(-1)
    out["y_pred"] = y_pred

elif task == "binary_classification":
    y_prob = np.asarray(y_raw).reshape(-1)
    y_class = (y_prob >= alpha).astype(int)
    out["y_prob"] = y_prob
    out["y_class"] = y_class

elif task == "multiclass_classification":
    y_scores = np.asarray(y_raw)

    if y_scores.ndim != 2 or y_scores.shape[1] != out_dim:
        raise ValueError(f"Salida inesperada para multiclase. shape={y_scores.shape}, out_dim={out_dim}")

    # 1) Asegurar "probabilidades": si no es softmax, construimos pseudo-prob con softmax estable
    if (act_name or "").lower().find("softmax") >= 0:
        y_prob = y_scores
    else:
        y_prob = stable_softmax(y_scores)

    # 2) Predicción por índice interno
    y_idx = y_prob.argmax(axis=1).astype(int)
    out["y_class_index"] = y_idx

    # 3) Mapear a clase original (si metadata lo permite)
    index_to_class, classes_original = get_index_to_class_from_meta(meta, out_dim)

    if index_to_class is not None:
        # OJO: v puede ser int o str; lo dejamos como viene
        y_class_original = np.array([index_to_class[int(i)] for i in y_idx], dtype=object)
        out["y_class"] = y_class_original
    else:
        # fallback: dejamos índice
        out["y_class"] = y_idx

    # 4) Confianza (prob del top-1)
    out["y_conf"] = y_prob.max(axis=1)

    # 5) Probabilidades por clase: nombrar con la clase original si existe, si no con índice
    if classes_original is not None:
        # ejemplo: prob_c1, prob_c2, prob_c3... (o prob_c5, prob_c7 en glass)
        for j, lab in enumerate(classes_original):
            out[f"prob_c{lab}"] = y_prob[:, j]
    else:
        for j in range(out_dim):
            out[f"prob_c{j}"] = y_prob[:, j]

    # 6) Top-3 (útil y barato)
    TOPK = 3 if out_dim >= 3 else out_dim
    top_idx = np.argsort(-y_prob, axis=1)[:, :TOPK]
    top_prob = np.take_along_axis(y_prob, top_idx, axis=1)

    if index_to_class is not None:
        # mapear índices top a etiquetas originales
        top_lab = np.vectorize(lambda i: index_to_class[int(i)], otypes=[object])(top_idx)
        for k in range(TOPK):
            out[f"top{k+1}_class"] = top_lab[:, k]
            out[f"top{k+1}_prob"] = top_prob[:, k]
    else:
        for k in range(TOPK):
            out[f"top{k+1}_class"] = top_idx[:, k]
            out[f"top{k+1}_prob"] = top_prob[:, k]

else:
    raise ValueError(f"Tarea no soportada: {task}")

out.to_csv(OUT_PRED, index=False)
print(f"\n✔ Predicciones guardadas en: {OUT_PRED}")
print(out.head())
